# 02_Chunking Optimization
Tips for Chunking Optimization to create search indexes for RAG

## Use case
- Azure AI Search の サービス仕様ドキュメントをインプットにする。
  - https://learn.microsoft.com/ja-jp/azure/search/
- ドキュメントは OCR が必要。
- 開発者マニュアルは、構造化されたセクションとなっている。
- 各セクションは非常に詳細かつ専門性の高い技術解説が記載されており、ドキュメントサイズも大きい。
- ドキュメントには、テキスト、テーブル、図、グラフなどが含まれるが、ここでは、テキスト、テーブルデータのみを扱う。

## チャンキング設計
- Document Intelligence で、Markdown形式でテキストデータを抽出済み。
- 1つのドキュメントに大量のコンテキストが含まれており、ドキュメントサイズも大きいため、チャンキングを実施する。
- ドキュメントは、技術要素ごとに明確なセクションわけがされており、各セクションで見るとLLMが扱えないレベルのデータサイズではない。そのため、セクション単位でチャンキングする。
- 各チャンキングのContentはEmbeddingする。
- Overlapping は行わない。
- 広い意味のコンテキストを保持するために、上位2つのヘッダー（Markdown形式：#, ##）をメタデータとして保持する。例えば、検索インデックスに関する記載があった場合に、それが「キーワード検索」に属する情報なのか、「ベクトル検索」に属する情報なのかを判断するために保持する。

In [1]:
! pip install python-dotenv langchain langchain-community langchain-openai langchainhub openai tiktoken azure-ai-documentintelligence azure-identity azure-search-documents==11.6.0b3

  Using cached azure_search_documents-11.6.0b4-py3-none-any.whl.metadata (23 kB)
Using cached azure_search_documents-11.6.0b4-py3-none-any.whl (325 kB)
  Attempting uninstall: azure-search-documents
    Found existing installation: azure-search-documents 11.6.0b3
    Uninstalling azure-search-documents-11.6.0b3:
      Successfully uninstalled azure-search-documents-11.6.0b3


In [2]:
"""
This code loads environment variables using the `dotenv` library and sets the necessary environment variables for Azure services.
The environment variables are loaded from the `.env` file in the same directory as this notebook.
"""
import os
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
doc_intelligence_endpoint = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT")
doc_intelligence_key = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_KEY")

In [ ]:
from langchain import hub
from langchain_openai import AzureChatOpenAI
from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader
from langchain_openai import AzureOpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.vectorstores.azuresearch import AzureSearch

### Load markdown file

In [ ]:
def read_file(filename):
    with open(filename, 'r', encoding="utf-8") as f:
        content = f.read()
    return content

In [ ]:
markdown_content = read_file('../output/azure-search-concept.md')
print(markdown_content)

# Azure Al Search の検索インデックス

[アーティクル]·2024/02/16

Azure Al Search の “検索インデックス"は検索可能なコンテンツであり、検索エンジン でインデックス作成、全文検索、ベクトル検索、ハイブリッド検索、フィルターされた クエリに使用できます。インデックスは、スキーマによって定義され、検索サービス に保存されます。2 番目のステップとしてデータのインポートが続きます。このコン テンツは検索サービス内に存在します。これは、最新の検索アプリケーションで想定さ れるミリ秒単位の応答時間に必要な、プライマリ データ ストアとは別のものです。イ ンデクサー主導のインデックス作成シナリオを除き、検索サービスがソース データに 接続したり、クエリを実行したりすることはありません。

検索インデックスを作成して管理する場合、この記事は次の点を理解するのに役立ちま す。

● コンテンツ(ドキュメントおよびスキーマ)

● 物理データ構造

● 基本操作

今すぐに使いたいですか?代わりに、検索インデックスの作成に関する記事を参照して ください。


## 検索インデックスのスキーマ

Azure Al Search のインデックスには 検索ドキュメントが格納されます。概念的に、ド キュメントはインデックス内で検索可能なデータの1 つの単位です。たとえば、小売 業者に製品ごとのドキュメントがあり、ニュース組織に記事ごとのドキュメントがある 場合、旅行サイトにはホテルと目的地ごとのドキュメントがある場合があります。こ れらの概念をなじみのあるデータベースの同等のものに対応させるなら、検索インデッ クスはテーブルと同じで、ドキュメントはテーブルにおける行とほぼ同じです。

次の例に示すように、ドキュメントの構造は“インデックス スキーマ"によって決まり ます。"フィールド"コレクションは通常、インデックスの最大の部分であり、各フィ ールドには、名前、データ型の割り当て、および使用方法を決定する許容される動作を 示す属性が設定されます。

JSON

{ "name": "name\_of\_index, unique across the service",

"fields": [

<!-- PageNumber="{" --

### Splitting with Langchain Text Splitter
https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/

We use `MarkdownHeaderTextSplitter` for markdown document.
  - https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/markdown_header_metadata/

In [ ]:
# Split the document into chunks base on markdown headers.
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
# Include the headers in the splits.
text_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)

splits = text_splitter.split_text(markdown_content)

print("Length of splits: " + str(len(splits)))

Length of splits: 311


In [13]:
splits

[Document(page_content='# Azure Al Search の検索インデックス  \n[アーティクル]·2024/02/16  \nAzure Al Search の “検索インデックス"は検索可能なコンテンツであり、検索エンジン でインデックス作成、全文検索、ベクトル検索、ハイブリッド検索、フィルターされた クエリに使用できます。インデックスは、スキーマによって定義され、検索サービス に保存されます。2 番目のステップとしてデータのインポートが続きます。このコン テンツは検索サービス内に存在します。これは、最新の検索アプリケーションで想定さ れるミリ秒単位の応答時間に必要な、プライマリ データ ストアとは別のものです。イ ンデクサー主導のインデックス作成シナリオを除き、検索サービスがソース データに 接続したり、クエリを実行したりすることはありません。  \n検索インデックスを作成して管理する場合、この記事は次の点を理解するのに役立ちま す。  \n● コンテンツ(ドキュメントおよびスキーマ)  \n● 物理データ構造  \n● 基本操作  \n今すぐに使いたいですか?代わりに、検索インデックスの作成に関する記事を参照して ください。', metadata={'Header 1': 'Azure Al Search の検索インデックス'}),
 Document(page_content='## 検索インデックスのスキーマ  \nAzure Al Search のインデックスには 検索ドキュメントが格納されます。概念的に、ド キュメントはインデックス内で検索可能なデータの1 つの単位です。たとえば、小売 業者に製品ごとのドキュメントがあり、ニュース組織に記事ごとのドキュメントがある 場合、旅行サイトにはホテルと目的地ごとのドキュメントがある場合があります。こ れらの概念をなじみのあるデータベースの同等のものに対応させるなら、検索インデッ クスはテーブルと同じで、ドキュメントはテーブルにおける行とほぼ同じです。  \n次の例に示すように、ドキュメントの構造は“インデックス スキーマ"によって決まり ます。"フィールド"コレクションは通常、インデックスの最大の部分であり、各フィ ールドには、名前、データ型の割り当て、および使用方法を決定する許容

## TBD: Chunking をミスっているところ
- Markdownの階層構造が元のドキュメントと異なっているチャンクがある。
  - 前後の情報から補正する？
- Imageの中のテキストも抽出している。
  - Imageは別途専用の処理を作って対応する。
- コードブロック（JSONなど）をヘッダーとして認識しているチャンクがある。
  - DIで読みった直後にルールベースの変換をはさむ
- 検索時には役に立たないリンクのみの情報が混じっている
  - 次のステップ
  - 関連項目

# 03_Index-Design
Tips for Azure AI Search Index-Design

### PUSH型
- シンプルなパターンとして、ローカルのデータを検索インデックスにインポートさせる。
- ユーザのコンテキストを捉えるために、ベクトル検索を採用する。そのため、Embedding フィールドを構成する。
  - Embedding Model: `text-embedding-ada-002`を採用する。
- ドキュメントのヘッダー情報を`Metadata`フィールドに含める。
- ドキュメントのタイトルを`Title`フィールドに含める。

In [14]:
from azure.search.documents.indexes.models import (
    ScoringProfile,
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    TextWeights,
)

In [63]:
aoai_embeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
    openai_api_version="2024-02-01",
)

vector_store_address: str = os.getenv("AZURE_SEARCH_ENDPOINT")
vector_store_password: str = os.getenv("AZURE_SEARCH_ADMIN_KEY")


fields = [
    SimpleField(
        name="id",
        type=SearchFieldDataType.String,
        key=True,
        filterable=True,
    ),
    SearchableField(
        name="content",
        type=SearchFieldDataType.String,
        searchable=True,
        analyzer_name="ja.microsoft"
    ),
    SearchField(
        name="content_vector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=len(aoai_embeddings.embed_query("Text")),
        vector_search_profile_name="myHnswProfile",
    ),
    SearchableField(
        name="metadata",
        type=SearchFieldDataType.String,
        searchable=True,
        analyzer_name="ja.microsoft"
    ),
    # Additional field to store the title
    SearchableField(
        name="title",
        type=SearchFieldDataType.String,
        searchable=True,
        analyzer_name="ja.microsoft"
    ),
]

index_name: str = "rag-search-index-0619"
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=aoai_embeddings.embed_query,
    fields=fields,
)

### 改善
不要な情報を検索インデックスに含めないようにする

In [64]:
# Remove the specific chapter from the index
keywords = ["関連項目", "次のステップ"]
for split in splits:
    if any(keyword in value for value in split.metadata.values() for keyword in keywords):
        print(split.metadata)
        continue
    vector_store.add_texts(
		[split.page_content],
  		[
        	{"title": f"{os.path.basename('../data/azure-search-concept.pdf')}"}
        ]
	)

{'Header 1': 'Azure Al Search の検索インデックス', 'Header 2': '次のステップ'}
{'Header 1': '基本的なベクトル ワークロードのフィールド コレクショ ン', 'Header 2': '関連項目'}
{'Header 1': 'demoblobstorage | Storage browser (preview) Storage account', 'Header 2': '次のステップ'}
{'Header 1': '関連項目'}
{'Header 1': '手順1:データ ソースを作成する', 'Header 2': '次のステップ'}
{'Header 1': 'ト', 'Header 2': '次のステップ'}
{'Header 1': 'データ ソースの検証チェックをバイパスする', 'Header 2': '次のステップ'}
{'Header 1': '次のステップ'}
{'Header 1': '次のステップ', 'Header 2': '最初のスキルセットを作成する'}
{'Header 1': '統合ベクター化の使用方法', 'Header 2': '次のステップ'}
{'Header 1': '1 注意', 'Header 2': '次のステップ'}
{'Header 1': '1 注意', 'Header 2': '関連項目'}
{'Header 1': '次のステップ'}
{'Header 1': 'Azure Al Search でベクトルやフルテキ ストを使用したハイブリッド検索', 'Header 2': '関連項目'}
{'Header 1': 'RAG シナリオの Azure Al Search クエリのコード例', 'Header 2': '関連項目'}
{'Header 1': '地理空間検索', 'Header 2': '次のステップ'}
{'Header 1': 'Azure Al Search でのセマンティック ラ ンク付け', 'Header 2': '関連項目'}
{'Header 1': 'スコアリング統計とスティッキー セッション', 'Header 2': '関連項目'}
{'Header 1': 'ベクトル検索での関連性', 'Header 2': 

# 04_Query-Design
Tips for Azure AI Search Query-Design

### Simple Search
- ユーザのクエリのコンテキストをとらえたい、かつサービスに特化したワードがクエリに含まれる可能性が高いため、フルテキスト検索＋ベクトル検索を組み合わせたHybrid検索を採用する。
  - Hybrid検索のスコアはAzure AI Searchでは、Reciprocal Rank Fusion (RRF) が採用される。 
- クエリはユーザのクエリをそのまま検索インデックスのクエリに利用する。

In [65]:
query="ベクトル検索時の設定要素について教えてください"
res_simple_query = vector_store.similarity_search(
    	query=query, k=3, search_type="hybrid"
)
res_simple_query

[Document(page_content='### 1 注意  \n2019 年1月1日より前に作成された一部の古い検索サービスは、ベクトル ワー クロードをサポートしないインフラストラクチャにデプロイされています。ベク :unselected: :unselected: トル フィールドをスキーマに追加しようとしてエラーが表示された場合、それは サービスが古いためです。このような場合は、ベクトル機能を試すために新しい 検索サービスを作成する必要があります。', metadata={'title': 'azure-search-concept.pdf'}),
 Document(page_content='## ベクトル検索の概念  \nベクトルを初めて使用する場合、このセクションではいくつかの主要な概念について説 明します。', metadata={'title': 'azure-search-concept.pdf'}),
 Document(page_content='## 1 注意  \n2019 年1月1日より前に作成された一部の古い検索サービスは、ベクトル ワー クロードをサポートしないインフラストラクチャにデプロイされています。ベク :unselected: :unselected: トル フィールドをスキーマに追加しようとしてエラーが表示された場合、それは サービスが古いためです。このような場合は、ベクトル機能を試すために新しい 検索サービスを作成する必要があります。', metadata={'title': 'azure-search-concept.pdf'})]

### Query Extensions
- ユーザのクエリのコンテキストをとらえたい、かつサービスに特化したワードがクエリに含まれる可能性が高いため、フルテキスト検索＋ベクトル検索を組み合わせたHybrid検索を採用する。
  - Hybrid検索のスコアはAzure AI Searchでは、Reciprocal Rank Fusion (RRF) が採用される。 
- また、ユーザクエリから検索クエリを新しく生成する。
  - クエリはユーザのクエリをスタンドアローンなクエリに変換する。
  - また、検索のカバレッジを大きくするために、類似した入力クエリを複数生成する。

In [66]:
import re
import os
from openai import AzureOpenAI
import json

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-02-01"
)

system_message = """
# Your Task
- Given the following conversation history and the users next question,rephrase the question to be a stand alone question.
- You also need to extend the original question to generate 5 related queries. This is done to capture the broader context of the user's question.
- You must output json format. In other words, You must output array of questions that length is 5.

# Json format example:
{
	"questions": [
		"related question 1",
		"related question 2",
		"related question 3",
		"related question 4",
		"related question 5"
	]
}
"""

def correct_text_gpt(text):
    message_text = [
		{"role":"system","content": system_message},
		{"role":"user","content": text}
	]
    completion = client.chat.completions.create(
		model="gpt-4o", # model = "deployment_name"
		messages = message_text,
		response_format={"type": "json_object"},
		temperature=0,
		)
    return completion.choices[0].message.content



In [67]:
query="ベクトル検索時の設定要素について教えてください"
expanded_query = correct_text_gpt(query)
parsed_data = json.loads(expanded_query)
parsed_data

{'questions': ['ベクトル検索時の設定要素にはどのようなものがありますか？',
  'ベクトル検索の設定要素を最適化する方法は何ですか？',
  'ベクトル検索の設定要素が検索結果に与える影響は何ですか？',
  'ベクトル検索の設定要素を変更する際の注意点は何ですか？',
  'ベクトル検索の設定要素を理解するためのリソースはありますか？']}

In [68]:
for question in parsed_data["questions"]:
	res_simple_query = vector_store.similarity_search(
		query=question, k=3, search_type="hybrid"
	)
	print(res_simple_query)
	print("\n")

[Document(page_content='## ベクトル検索について  \nベクトル検索は、ドキュメントとクエリがプレーン テキストではなくベクトルとして 表現される場合の情報取得の方法です。ベクトル検索では、機械学習モデルがソース 入力(テキスト、画像、その他のコンテンツ) のベクトル表現を生成します。コンテン :unselected: ツの数学表現を使用することによって、検索シナリオの共通基盤が提供されます。 す べてがベクトルであれば、関連する元のコンテンツがクエリとは異なるメディアや言語 であっても、クエリはベクトル空間で一致するものを見つけることができます。', metadata={'title': 'azure-search-concept.pdf'}), Document(page_content='## ベクトル検索を使用する理由  \n検索可能なコンテンツがベクトルとして表されると、クエリは類似するコンテンツ内の 近い一致を見つけることができます。ベクトル生成に使用される埋め込みモデルは、 どの単語と概念が類似しているかを認識し、結果のベクトルを埋め込み空間内で近くに 配置します。たとえば、"クラウド"と"霧" に関するベクトル化されたソース ドキュメ ントは、意味的に類似しているため、構文上の一致ではない場合も“霧" に関するクエ リで表示される可能性が高くなります。', metadata={'title': 'azure-search-concept.pdf'}), Document(page_content='## ベクトル検索の概念  \nベクトルを初めて使用する場合、このセクションではいくつかの主要な概念について説 明します。', metadata={'title': 'azure-search-concept.pdf'})]


[Document(page_content='# ベクトル検索での関連性  \n[アーティクル]·2024/04/18  \nベクトル クエリの実行時、検索エンジンは類似のベクトルを検索して、検索結果に返 される最適な候補を見つけます。ベクトル コンテンツのインデックス付け方法に応じ て、関連する一致の検索は網羅的であるか、ニアネイバーに制限されて処理が高速化さ れます。候補が見つかると、類似性メトリッ

### LangChain Query Expansion

In [69]:
from langchain_core.pydantic_v1 import BaseModel, Field


class ParaphrasedQuery(BaseModel):
    """You have performed query expansion to generate a paraphrasing of a question."""

    paraphrased_query: str = Field(
        ...,
        description="A unique paraphrasing of the original question.",
    )

In [70]:
from langchain.output_parsers import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

system = """You are an expert at converting user questions into database queries. \
You have access to a database of tutorial videos about a software library for building LLM-powered applications. \

Perform query expansion. If there are multiple common ways of phrasing a user question \
or common synonyms for key words in the question, make sure to return multiple versions \
of the query with the different phrasings.

If there are acronyms or words you are not familiar with, do not try to rephrase them.

Return at least 3 versions of the question."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
AzureChatOpenAI
llm = AzureChatOpenAI(
	openai_api_version="2024-02-01",
    azure_deployment="gpt-4o",
)
llm_with_tools = llm.bind_tools([ParaphrasedQuery])
query_analyzer = prompt | llm_with_tools | PydanticToolsParser(tools=[ParaphrasedQuery])

In [72]:
expanded_query = query_analyzer.invoke(
    {
        "question": query,
    }
)
expanded_query

[ParaphrasedQuery(paraphrased_query='ベクトル検索の設定要素について教えてください'),
 ParaphrasedQuery(paraphrased_query='ベクトル検索を設定する際の要素を教えてください'),
 ParaphrasedQuery(paraphrased_query='ベクトル検索の設定項目について教えてください')]

In [73]:
for question in expanded_query:
	res_simple_query = vector_store.similarity_search(
		query=question.paraphrased_query, k=3, search_type="hybrid"
	)
	print(res_simple_query)
	print("\n")

[Document(page_content='### 1 注意  \n2019 年1月1日より前に作成された一部の古い検索サービスは、ベクトル ワー クロードをサポートしないインフラストラクチャにデプロイされています。ベク :unselected: :unselected: トル フィールドをスキーマに追加しようとしてエラーが表示された場合、それは サービスが古いためです。このような場合は、ベクトル機能を試すために新しい 検索サービスを作成する必要があります。', metadata={'title': 'azure-search-concept.pdf'}), Document(page_content='## ベクトル検索の概念  \nベクトルを初めて使用する場合、このセクションではいくつかの主要な概念について説 明します。', metadata={'title': 'azure-search-concept.pdf'}), Document(page_content='## 1 注意  \n2019 年1月1日より前に作成された一部の古い検索サービスは、ベクトル ワー クロードをサポートしないインフラストラクチャにデプロイされています。ベク :unselected: :unselected: トル フィールドをスキーマに追加しようとしてエラーが表示された場合、それは サービスが古いためです。このような場合は、ベクトル機能を試すために新しい 検索サービスを作成する必要があります。', metadata={'title': 'azure-search-concept.pdf'})]


[Document(page_content='### 1 注意  \n2019 年1月1日より前に作成された一部の古い検索サービスは、ベクトル ワー クロードをサポートしないインフラストラクチャにデプロイされています。ベク :unselected: :unselected: トル フィールドをスキーマに追加しようとしてエラーが表示された場合、それは サービスが古いためです。このような場合は、ベクトル機能を試すために新しい 検索サービスを作成する必要があります。', metadata={'title': 'azure-search-concept.pdf'}), Document(

### HyDE (Hypothetical Document Embeddings)

In [76]:
import re
import os
from openai import AzureOpenAI
import json

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-02-01"
)


def correct_text_gpt(text):
    hypothetical_gen_instruction = f"""Please write a passage to answer the question
	Question: {text}
	Passage:
	"""
    message_text = [
		{"role":"system","content": "You are an AI assistant."},
		{"role":"user","content": hypothetical_gen_instruction}
	]
    completion = client.chat.completions.create(
		model="gpt-4o", # model = "deployment_name"
		messages = message_text,
		# response_format={"type": "json_object"},
		temperature=0,
		)
    return completion.choices[0].message.content



In [77]:
query="ベクトル検索時の設定要素について教えてください"
hypothetical_answer = correct_text_gpt(query)
hypothetical_answer

'ベクトル検索時の設定要素について説明します。ベクトル検索は、データベース内のベクトルデータを効率的に検索するための手法です。以下の主要な設定要素があります。\n\n1. **距離測定方法**: ベクトル間の類似度を測定するための方法です。一般的な距離測定方法には、ユークリッド距離、コサイン類似度、マンハッタン距離などがあります。選択する距離測定方法は、データの特性や検索の目的に応じて異なります。\n\n2. **次元数**: ベクトルの次元数は、データの特徴をどれだけ詳細に表現するかを決定します。次元数が多いほど、より詳細な情報を保持できますが、計算コストも増加します。適切な次元数を選定することが重要です。\n\n3. **インデックス構築方法**: ベクトル検索の効率を向上させるために、インデックスを構築する方法です。主なインデックス構築方法には、KDツリー、ボールツリー、LSH（局所感度ハッシュ）などがあります。データの特性や検索速度の要件に応じて、適切なインデックス構築方法を選択します。\n\n4. **検索アルゴリズム**: ベクトル検索に使用するアルゴリズムです。一般的なアルゴリズムには、近似最近傍探索（ANN）や正確な最近傍探索があります。ANNは高速ですが、結果の精度が若干低下する可能性があります。一方、正確な最近傍探索は精度が高いですが、計算コストが高くなります。\n\n5. **パラメータ調整**: 検索の精度と速度を最適化するためのパラメータ調整です。例えば、ANNを使用する場合、探索の深さや候補数などのパラメータを調整することで、検索のパフォーマンスを向上させることができます。\n\nこれらの設定要素を適切に調整することで、ベクトル検索の効率と精度を最大化することができます。データの特性や検索の目的に応じて、最適な設定を選定することが重要です。'

In [78]:
res_simple_query = vector_store.similarity_search(
		query=hypothetical_answer, k=3, search_type="hybrid"
	)
res_simple_query

[Document(page_content='## ニアレストネイバー検索  \nベクトル検索では、検索エンジンは埋め込みスペース内のベクトルをスキャンして、ク エリ ベクトルに最も近いベクトルを識別します。この手法は“ニアレストネイバー検 索" と呼ばれます。△ ニアレストネイバーは、項目間の類似性を定量化するのに役立ち ます。ベクトルの類似性が高い場合は、元のデータも同様であることを示します。高 速なニアレストネイバー検索を容易にするために、検索エンジンでは最適化を実行する か、データ構造およびデータ パーティション分割を使用して検索領域を削減します。  \n各ベクトル検索アルゴリズムは、最小待機時間、最大スループット、再現率、メモリを 最適化する際に、ニアレストネイバーの問題をさまざまな方法で解決します。類似性 を計算するために、類似性メトリックでは距離を計算するためのメカニズムを提供しま す。  \nAzure Al Search では現在、次のアルゴリズムがサポートされています。  \n● Hierarchical Navigable Small World (HNSW): HNSW は、データ分散が不明である か、頻繁に変更される可能性がある、高いリコールと待機時間の短い用途に最適 化された主要な ANN アルゴリズムです。高次元のデータ ポイントを階層グラフ 構造に整理することで、高速でスケーラブルな類似性検索を可能にしながら、検 索精度と計算コストのトレードオフを調整できます。このアルゴリズムでは、高 速ランダム アクセスのためにすべてのデータ ポイントがメモリ内に存在する必要  \nがあるため、このアルゴリズムではベクトル インデックス サイズのクォータが使 用されます。  \n● 完全な K ニアレストネイバー (KNN): クエリ ベクトルとすべてのデータ ポイント の間の距離を計算します。計算負荷が高いので、小規模なデータセットに最適で す。このアルゴリズムではデータ ポイントの高速ランダム アクセスが不要なた め、このアルゴリズムではベクトル インデックス サイズのクォータが使用されま せん。ただし、このアルゴリズムではニアレストネイバーのグローバル セットが :unselected: 提供されます。  \nインデックス定義内で1つ以上の